In [39]:
%matplotlib inline
from WindPy import *
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib
from sklearn.metrics import precision_recall_curve  
from sklearn.metrics import classification_report  
import pandas as pd
import datetime as dt 
from sklearn import tree
from sklearn.svm import SVC
from sklearn import preprocessing

def download_data(code,s_date,e_date):
    w.start()
    #code = '000600.SZ'
    factor = "close,volume,turn ,RSI,MACD"
    data = w.wsd(code, factor, s_date, e_date,"RSI_N=6;MACD_L=26;MACD_S=12;MACD_N=9;MACD_IO=1;Fill=Previous;PriceAdj=F")  
    df_data = {}
    if data.ErrorCode == 0:
        dfc = dict(zip(data.Fields,data.Data))
        #df_data = pd.DataFrame(dfc,index=data.Times)
        df_data = pd.DataFrame(dfc)
        df_data = df_data.dropna()
        #df_data['code'] = code
        #df_data.index = df_data.index.strftime('%Y-%m-%d')
    else: 
        print(data)
    return df_data

def normalized(df):
    
    return df

def prepare_data(dataframe,nor = True):
    a =dataframe.values
    #print(a)

    x=[]
    y=[]
    for i in range(len(a)-1):
        if a[i][0]<a[i+1][0]:
            y.append(1)
        else:
            y.append(0)
    
    xa = dataframe
    # normalize the data attributes
    #normalized_X = preprocessing.normalize(b.values)
    #print("normalized : ",normalized_X)
    # standardize the data attributes
    #standardized_X = preprocessing.scale(b.values)
    #print("standardized: " ,standardized_X)
    
    
    #标准化
    if nor == True:
        xa = (dataframe - dataframe.mean())/ dataframe.std()
    #print(xa)
    x = list(xa.values[1:])
    for i in range(len(x)):
        x[i] = np.round(x[i],1)    
    return x, y

##训练模型
def model_train(code,s_date,e_date):
    #data1=DataAPI.MktEqudGet(secID=universe,beginDate=begin_date,endDate=end_date,field=['tradeDate','highestPrice','lowestPrice','openPrice','closePrice','turnoverVol','turnoverRate'],pandas="1")
    #data2=DataAPI.MktStockFactorsDateRangeGet(secID=universe[0],beginDate=begin_date,endDate=end_date,field=['tradeDate','DAVOL5','EMA5','EMA10','MA5','MA10','RSI','VOL5','VOL10','MACD'],pandas="1")  
    
    df_data = download_data(code,s_date,e_date)
    x,y = prepare_data(df_data)
    #print(x)
    #print(y)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.0, random_state=42)

    #print(x_train)
    ##训练模型   
    #print(y_train)
    ''' 使用信息熵作为划分标准，对决策树进行训练 criterion=entropy '''
    clf = tree.DecisionTreeClassifier( max_depth =3 )
    clf.fit(x_train,y_train)
    
    #clf = SVC()
    #clf.fit(x_train,y_train)
    
    
    print("feature 权重:  ",clf.feature_importances_)  
    
    y_predict=clf.predict(x_train)
    
    #准确率与召回率  
    #precision, recall, thresholds = precision_recall_curve(y_train, y_predict)  
    #print("统计: ",precision, recall, thresholds)
    #report = clf.predict_proba(x) # 某一个值预测概率
    #print("report: ",report)
    print(classification_report(y_train, y_predict,target_names = ['1', '0']))  
    
    #print(y_predict)
    correct_rate,recall_rate,balance = analy(y_train,y_predict)
    joblib.dump(clf, 'clf.model')
    
    return clf,correct_rate,recall_rate ,balance

#统计样本##正确率、召回率、正负样本均衡度
def analy(y_train,y_predict):

    n_1=0
    for i in range(len(y_predict)):
        if y_train[i]==y_predict[i]:
            n_1=n_1+1
    n_2=0
    for i in range(len(y_predict)):
        if y_train[i]==y_predict[i] and y_predict[i]==1:
            n_2=n_2+1
    correct_rate = float(n_1)/float( len(y_predict) )
    recall_rate = float(n_2)/float( int(sum(y_train)) )
    balance = float(sum(y_train))/float(len(y_train))
    
    return correct_rate,recall_rate,balance

def initialize(code):
    #code  = '000600.SZ'
    today = dt.date.today() - dt.timedelta(days=30)
    sdate = dt.date(today.year -1 , today.month,today.day).isoformat()
    edate = dt.date(today.year , today.month,today.day).isoformat()
    print("train data : from ",sdate, " to " , edate)
    model,acc_rate,recall_rate,balance=model_train(code,sdate,edate)
    #print("样本正确率、召回率、正负样本均衡度: ")
    #print(acc_rate,recall_rate,balance)  ##正确率、召回率(预测对的正样本数)、正负样本均衡度
    return model



In [41]:
#data = w.wsd('000600.SZ', "high,low,open,close,volume,turn,MA,RSI,VMA,MACD", '2015-12-31', '2015-12-31',"MA_N=10;RSI_N=6;VMA_N=10;MACD_L=26;MACD_S=12;MACD_N=9;MACD_IO=1;Fill=Previous") 
#data
 
code  = '000656.SZ'
clf = initialize(code)
#print(clf)
#clf.predict_proba(x)

b = download_data(code,'20151210','20160105')
print(b)

xb,yb = prepare_data(b)
#print("预处理: ",xb)

print(yb)

p_y = clf.predict(xb)
print(list(p_y))

analy(yb,p_y)



train data : from  2014-12-08  to  2015-12-08
feature 权重:   [ 0.          0.20657258  0.68700776  0.10641966  0.        ]
             precision    recall  f1-score   support

          1       0.78      0.79      0.78       132
          0       0.75      0.74      0.74       112

avg / total       0.77      0.77      0.77       244

    CLOSE      MACD        RSI      TURN     VOLUME
0    5.38 -0.010133  32.542440  0.979553   29233683
1    5.30 -0.036801  29.319363  0.906943   27066723
2    5.44 -0.046108  41.488993  0.950355   28362312
3    5.51 -0.047291  46.967592  1.074685   32072786
4    5.50 -0.048476  46.225603  1.106803   33031334
5    5.67 -0.035290  59.331980  2.241963   66908914
6    5.64 -0.026951  56.419965  1.815305   54175772
7    5.94  0.003821  72.573236  4.628930  138145329
8    5.92  0.026292  70.483225  2.891803   86429904
9    5.56  0.014879  43.453153  7.834259  285855639
10   5.40 -0.006995  36.074734  4.426582  161516674
11   5.43 -0.021660  38.427148  1.95333

(0.7058823529411765, 0.5, 0.35294117647058826)